## 1. В чем принципиальные отличия item-item рекомендаций от ALS?

##### ответ:

- __item-item recommender__ - это аналог алгоритма __KNN__, то есть алгоритма ближайших соседей, но модифицированный на очень большие расстояния между объектами. Если объекты попадают между средним значением соседей, то они считаются схожими. Учит рейтинг.
- __ALS__ - этот алгоритм находит приближенные разложения изначальной - разреженной матрицы, вместо того, что бы искать объекты в этой матрице напрямую, по сути уменьшает размерность изначальной матрицы разбивая ее на 2 более узкие в которой количество взаимодействий и не взаимодействий 50/50%. Является алгоритмом оптимизации, для поиска значений по координатам векторов. Веса будут зависеть от координат настраиваемых вручную и фактическим расположением факта взаимодействия пользователя с товаром. Учит факт взаимодействия, чем он чаще, тем вес ошибки больше. Очень быстрый алгоритм.

## 2. Приведите 3 примера весов user-item матрицы для задачи рекоммендаций товаров

##### ответ:
- Взаимодействие пользователя с кантентом в соц сетях: нравится - не нравится, репост - или нет, коммент - или нет.
- Факт покупки товара: купил первый товар или второй, добавил в карзину первый товар или второй.
- Просмотр страничек или посещение сайтов переходы по ссылкам: прошол по ссылке на сайт или нет.
- Фильстрация писем: добавил письмо в спам или нет.

## 3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

##### ответ:
- Считает лишь факт взаимодействия пользователя с товаром, но веса зависят от количества взаимодействий.
- Строковые значения передавать нельзя, если есть строковые значения их надо рекодировать в категории целочисленного типа
- После уменьшения размерности матрици на 2 более узких, обратно целую матрицу восстановить уже нельзя.

## 4. Мы рассматривали bm_25_weight. Опишите, как оно работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки)

- Взвешивает каждую отдельную строку разреженной(изначальной) матрицы пользователь-предмет взвешиванием BM25.

##### Ниже приведен код раскомментированный код взятый со странички git hub

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix

In [2]:
test = np.arange(12).reshape(3,4)
test

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [10]:
def bm25_weight(X, K1=100, B=0.8):
    """ Weighs each row of a sparse matrix X  by BM25 weighting """
    
    # calculate idf per term (user)
    X = coo_matrix(X) # матрица переводится в формат матрицы типа np.array, но модифицированный под scipy

    N = float(X.shape[0])  # принимаются строки матрицы в формате float
    binc = np.bincount(X.col) # подсчитывается количество вхождений каждого объекта в строке
    idf = np.log(N) - np.log(1 + binc) # из логарифма по количеству строк матрицы
                                 # вычитается логарифт 1 + количество каждого объекта в строке

    # calculate length_norm per document (artist)
    X_sum = X.sum(axis=1)      # суммтся значения столбцов
    row_sums = np.ravel(X_sum) # переводятся в одномерный массив
    average_length = row_sums.mean() # выводится среднее значение
    length_norm = (1.0 - B) + B * row_sums / average_length # нормализация строк матрицы
                                                            # одномерный массив делится на его среднее значение
                                                            # после этого умножается на дефолтный вес 0.8 + 0.2
    
    # weight matrix rows by bm25
    X.data = X.data * (K1 + 1.0) / (K1 * length_norm[X.row] + X.data) * idf[X.col] # здесь происходит перевзвешивание 
                                                                                   # вcей матрицы
        
    return X, idf, length_norm

In [11]:
X, idf, length_norm = bm25_weight(test)

In [12]:
X.data

array([-0.12030341, -0.12030341, -0.12030341,  0.        , -0.28768207,
       -0.28768207, -0.28768207,  0.        , -0.45506073, -0.45506073,
       -0.45506073])

### Источники

- [код implisit - сайт: git hub - автор: Бен Фредериксон](https://github.com/benfred/implicit/blob/master/implicit/nearest_neighbours.py)
- [Сайт с формулой BM25](https://seonomad.net/article/algoritm-bm25)